In [1]:
import pandas as pd
import sqlite3
import os

In [2]:
db_path = "my_database.db"  # SQLite database file
conn = sqlite3.connect(db_path)


In [3]:
from sqlalchemy import create_engine

engine = create_engine(f"sqlite:///{db_path}")

#### Quarter over quarter trend

In [4]:
query = """
with QoQ as (SELECT strftime('%Y', close_date) AS year,
    (cast(strftime('%m', close_date) as int)-1)/3 + 1 AS quarter,
sum(case when a.deal_stage='Won' then a.close_value else 0 end) as total_closed_value,
count(case when deal_stage='Won' then 1 end) as Number_of_Deals_Won, count(*) total_deals,
round(count(case when deal_stage='Won' then 1 end)*100.0/count(*),2) as win_rate,
sum(case when a.deal_stage='Won' then a.close_value else 0 end)/count(case when deal_stage='Won' then 1 end) as avg_deal_size
FROM sales_pipelines a join sales_teams b
on a.sales_agent=b.sales_agent
where quarter is not null
group by quarter)
 SELECT year, quarter,
 total_closed_value,
ROUND(100.0 * (total_closed_value - LAG(total_closed_value) OVER (ORDER BY year, quarter))
/ NULLIF(LAG(total_closed_value) OVER (ORDER BY year, quarter), 0), 2) AS revenue_pct
from QoQ

"""
quarter_sales = pd.read_sql(query, conn)
print(quarter_sales)

   year  quarter  total_closed_value  revenue_pct
0  2017        1           1134672.0          NaN
1  2017        2           3086111.0       171.98
2  2017        3           2982255.0        -3.37
3  2017        4           2802496.0        -6.03


#### Revenue in 2017 grew aggressively in Q2 (+172%), but the following two quarters showed negative growth (-3.37% in Q3 and -6.03% in Q4), suggesting that the Q2 boost was exceptional and not sustained. This indicates strong short-term gains but highlights a need for consistent pipeline management to avoid sharp slowdowns.

#### performance of the sales_agent 

In [5]:
query = """
with TEAM as (SELECT a.sales_agent,regional_office,
sum(case when a.deal_stage='Won' then a.close_value else 0 end) as total_closed_value,
count(case when deal_stage='Won' then 1 end) as Number_of_Deals_Won, count(*) total_deals,
round(count(case when deal_stage='Won' then 1 end)*100.0/count(*),2) as win_rate,
round(sum(case when a.deal_stage='Won' then a.close_value else 0 end)/count(case when deal_stage='Won' then 1 end),2) as avg_deal_size
FROM sales_pipelines a join sales_teams b
on a.sales_agent=b.sales_agent
group by a.sales_agent)
SELECT *,
    RANK() OVER (ORDER BY total_closed_value DESC) AS revenue_rank,
    RANK() OVER (ORDER BY Number_of_Deals_Won DESC) AS deals_won_rank,
    RANK() OVER (ORDER BY win_rate DESC) AS win_rate_rank,
    RANK() OVER (ORDER BY avg_deal_size DESC) AS avg_deal_size_rank
FROM TEAM
ORDER BY revenue_rank;
"""
sales_by_sales_agent = pd.read_sql(query, conn)
print(sales_by_sales_agent)

           sales_agent regional_office  total_closed_value  \
0      Darcel Schlecht         Central           1153214.0   
1       Vicki Laflamme            West            478396.0   
2      Kary Hendrixson            West            454298.0   
3         Cassey Cress            East            450489.0   
4        Donn Cantrell            East            445860.0   
5         Reed Clapper            East            438336.0   
6            Zane Levy            West            430068.0   
7        Corliss Cosme            East            421036.0   
8       James Ascencio            West            413533.0   
9      Daniell Hammack            East            364229.0   
10     Maureen Marcano            West            350395.0   
11    Gladys Colclough         Central            345674.0   
12      Markita Hansen            West            328792.0   
13       Kami Bicknell            West            316456.0   
14   Marty Freudenburg         Central            291195.0   
15      

#### Sales agent performance analysis shows that Darcel Schlecht drives the highest revenue through sheer volume of deals, while Reed Clapper stands out with the best efficiency (65% win rate). Elease Gluck specializes in high-value deals, though with lower volume. In contrast, Versie Hillebrand and Violet Mclelland underperform, winning deals but with low overall revenue contribution due to small deal sizes. This highlights the need to scale efficient closers, leverage high-value specialists and Balancing teams:Pair volume closers (Darcel) with efficiency closers (Reed) in mentoring/shadowing setups.

#### performance of sales_team

In [38]:
query = """
with TEAM as (SELECT manager,regional_office,
sum(case when a.deal_stage='Won' then a.close_value else 0 end) as total_closed_value,
count(case when deal_stage='Won' then 1 end) as Number_of_Deals_Won, count(*) total_deals,
round(count(case when deal_stage='Won' then 1 end)*100.0/count(*),2) as win_rate,
round(sum(case when a.deal_stage='Won' then a.close_value else 0 end)/count(case when deal_stage='Won' then 1 end),2) as avg_deal_size
FROM sales_pipelines a join sales_teams b
on a.sales_agent=b.sales_agent
group by b.manager)
SELECT *,
    RANK() OVER (ORDER BY total_closed_value DESC) AS revenue_rank,
    RANK() OVER (ORDER BY Number_of_Deals_Won DESC) AS deals_won_rank,
    RANK() OVER (ORDER BY win_rate DESC) AS win_rate_rank,
    RANK() OVER (ORDER BY avg_deal_size DESC) AS avg_deal_size_rank
FROM TEAM
ORDER BY revenue_rank;
"""
sales_by_team = pd.read_sql(query, conn)
print(sales_by_team)

            manager regional_office  total_closed_value  Number_of_Deals_Won  \
0     Melvin Marxen         Central           2251930.0                  882   
1     Summer Sewald            West           1964750.0                  828   
2     Rocco Neubert            East           1960545.0                  691   
3      Celia Rouche            West           1603897.0                  610   
4        Cara Losch            East           1130049.0                  480   
5  Dustin Brinkmann         Central           1094363.0                  747   

   total_deals  win_rate  avg_deal_size  revenue_rank  deals_won_rank  \
0         1929     45.72        2553.21             1               1   
1         1701     48.68        2372.89             2               2   
2         1327     52.07        2837.26             3               4   
3         1296     47.07        2629.34             4               5   
4          964     49.79        2354.27             5               6   
5

#### Melvin Marxen is the top revenue contributor, but their low win rate . If they can improve closing effectiveness, they could not only dominate in revenue but also drive sustainable growth for the team.
#### Rocco Neubert  performance highlights quality over quantity — making him the most reliable closer for high-value opportunities.
#### Dustin Brinkmann performance suggests a focus on smaller deals with limited impact. By improving qualification and targeting larger opportunities, Dustin could enhance both efficiency and revenue contribution.

#### product with good win_rate

In [44]:
query = """
SELECT a.product,
round(count(case when deal_stage='Won' then 1 end)*100.0/count(*),2) as win_rate
FROM sales_pipelines a join sales_teams b
on a.sales_agent=b.sales_agent
group by a.product
order by win_rate desc
"""
sales_by_sales_agent = pd.read_sql(query, conn)
print(sales_by_sales_agent)

          product  win_rate
0    GTX Plus Pro     49.48
1          GTXPro     49.26
2       GTX Basic     49.04
3      MG Special     48.03
4  GTX Plus Basic     47.22
5     MG Advanced     46.32
6         GTK 500     37.50


#### Product analysis revealed that the GTX line (especially GTX Plus Pro, GTXPro, and GTX Basic) consistently achieved ~50% win rates, indicating strong customer acceptance. In contrast, MG Advanced and particularly GTK 500 underperformed, with GTK 500 showing the weakest win rate at 37.5%. This highlights a need for strategic review (pricing, customer feedback, competitor comparison) to understand why they underperform, while leveraging the GTX series as proven market leaders.”